In [1]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama2-finetune/train.sh
/kaggle/input/llama2-finetune/requirements.txt
/kaggle/input/llama2-finetune/scripts/train.py
/kaggle/input/llama2-finetune/dataset/prompter.py
/kaggle/input/llama2-finetune/dataset/utils.py
/kaggle/input/llama2-finetune/dataset/__init__.py
/kaggle/input/llama2-finetune/argument/argument_class.py
/kaggle/input/llama2-finetune/data/train.json
/kaggle/input/llama2-finetune/data/test.json
/kaggle/input/llama2-finetune/data/alpaca.json
/kaggle/input/llama2-finetune/data/eval.json


In [14]:
# first time run
# set persistence to files only

# !cp -r /kaggle/input/llama2-finetune /kaggle/working

In [2]:
cd llama2-finetune

/kaggle/working/llama2-finetune


In [3]:
!pip install -q -r requirements.txt

In [4]:
!pip install -e .

Obtaining file:///kaggle/working/llama2-finetune
  Preparing metadata (setup.py) ... done
  Running setup.py develop for Llama2ViFinetuning


In [ ]:
!TORCH_DISTRIBUTED_DEBUG="INFO" \
CUDA_VISIBLE_DEVICES="0, 1" \
python ./scripts/train.py \
  --model_name_or_path "NousResearch/Llama-2-7b-hf"\
  --train_data_file ./data/train.json \
  --eval_data_file ./data/eval.json \
  --do_train True \
  --no_cuda False \
  --per_device_train_batch_size 4 \
  --per_device_eval_batch_size 1 \
  --output_dir ./checkpoint/vi-finetune-v1 \
  --overwrite_output_dir True \
  --weight_decay 0.001 \
  --learning_rate 2e-4 \
  --warmup_ratio 0.05 \
  --evaluation_strategy steps \
  --gradient_accumulation_step 2 \
  --group_by_length True \
  --max_steps 10000 \
  --max_grad_norm 0.3 \
  --logging_strategy steps \
  --save_strategy steps \
  --logging_steps 200 \
  --save_steps 1000 \
  --eval_steps 2000 \
  --report_to tensorboard \
  --lr_scheduler_type cosine \
  --save_total_limit 3 \
  --fp16 False \
  --bf16 False \
  --prompt_path ./data/alpaca.json \
  --optim paged_adamw_32bit

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Loading checkpoint shards: 100%|██████████████████| 2/2 [01:08<00:00, 34.01s/it]
This is file name ./data/alpaca.json
Extracting data files: 100%|█████████████████████| 1/1 [00:00<00:00, 123.40it/s]
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-76dfe03736cd51d8/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.
Extracting data files: 100%|████████████████████| 1/1 [00:00<00:00, 1006.07it/s]
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-039e418d36b1d802/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.
100%|███████████████████████████████████

# Inference

In [5]:
model_name =  "NousResearch/Llama-2-7b-hf"
device_map = "auto"
# new_model = 

In [6]:
import os
from functools import partial

import torch
from trl import SFTTrainer
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig, set_peft_model_state_dict, PeftModel

from argument.argument_class import DataTrainingArguments, ModelArguments, CustomTrainingArguments
from dataset import Prompter, generate_prompt

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

In [8]:
checkpoint_path = "./checkpoint/vi-finetune-v1/checkpoint-1000"
model = PeftModel.from_pretrained(base_model, checkpoint_path)

In [9]:
model = model.merge_and_unload()

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [15]:
from functools import partial

prompter = Prompter("alpaca", template_json_path="./data/alpaca.json")
generate_prompt = partial(generate_prompt, prompter=prompter)

This is file name ./data/alpaca.json


In [17]:
from transformers import GenerationConfig

In [47]:
generation_config = GenerationConfig(
    max_new_tokens=512, 
    early_stopping=True, 
    do_sample=True,
    num_beams=3,
    use_cache=True,
    temperature=0.1,
    top_k=40,
#     top_p=0.75,
    repetition_penalty=2.0,
    remove_invalid_values=True,
    renormalize_logits=True
)

In [24]:
prompt = "Dựa vào đoạn văn sau, hãy trả lời câu hỏi sau một cách ngắn gọn nhất"
input = """Đoạn văn: 'Hồ Chí Minh (chữ Nho: 胡志明; 19 tháng 5 năm 1890 – 2 tháng 9 năm 1969), tên khai sinh là Nguyễn Sinh Cung (chữ Nho: 阮生恭), còn được biết với tên gọi Bác Hồ, là một nhà cách mạng và chính khách người Việt Nam. Ông là người sáng lập Đảng Cộng sản Việt Nam, từng là Chủ tịch nước Việt Nam Dân chủ Cộng hoà từ 1945–1969, Thủ tướng Việt Nam Dân chủ Cộng hòa trong những năm 1945–1955, Tổng Bí thư Ban Chấp hành Trung ương Đảng Lao động Việt Nam từ 1956–1960, Chủ tịch Ban Chấp hành Trung ương Đảng Lao động Việt Nam từ năm 1951 cho đến khi qua đời.'
 Câu hỏi: Hồ Chí Minh có tên khai sinh và tên chữ nho là gì?"""
example = {"instruction": prompt, "input": input, "output": ""}
prompt = generate_prompt(example)

In [19]:
input_ids = tokenizer(prompt["text"], return_tensors="pt").input_ids
output = model.generate(input_ids=input_ids, generation_config=generation_config)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [20]:
tokenizer.decode(output[0])

"<s> Dưới đây là một hướng dẫn mô tả một nhiệm vụ, kèm với thông tin cung cấp thêm bối cảnh.Hãy viết một phản hồi phù hợp để hoàn thành yêu cầu đó.\n\n### Chỉ dẫn:\nDựa vào đoạn văn sau, hãy trả lời câu hỏi sau một cách ngắn gọn nhất\n\n### Thông tin:\nĐoạn văn: Dựa vào đoạn văn sau: 'Hồ Chí Minh (chữ Nho: 胡志明; 19 tháng 5 năm 1890 – 2 tháng 9 năm 1969), tên khai sinh là Nguyễn Sinh Cung (chữ Nho: 阮生恭), còn được biết với tên gọi Bác Hồ, là một nhà cách mạng và chính khách người Việt Nam. Ông là người sáng lập Đảng Cộng sản Việt Nam, từng là Chủ tịch nước Việt Nam Dân chủ Cộng hoà từ 1945–1969, Thủ tướng Việt Nam Dân chủ Cộng hòa trong những năm 1945–1955, Tổng Bí thư Ban Chấp hành Trung ương Đảng Lao động Việt Nam từ 1956–1960, Chủ tịch Ban Chấp hành Trung ương Đảng Lao động Việt Nam từ năm 1951 cho đến khi qua đời.'\n Câu hỏi: Hồ Chí Minh có tên khai sinh và tên chữ nho là gì?\n\n### Phản hồi:\nTên khai sinh của Hồ Chí Minh là Nguyễn Sinh Cung (chữ Nho: 阮生恭). Tên chữ Nho của ông là Bác

In [49]:
prompt = "Dựa vào thông tin sau, hãy trả lời câu hỏi sau một cách ngắn gọn nhất. Không được phép xin lỗi"
input = """
Thông tin:
Team CV có các sản phẩm như sau:
- Phân tích hình ảnh bao gồm việc nhận diện các vật thể trong hình và đưa ra các thông tin bao gồm: chủ thể, sản phẩm (áo, quần, dây nịt, ...) và các thông tin của sản phẩm như (màu sắc, kiểu dáng, ...)
- Lõi nhận diện chữ viết in là để nhằm trích xuất thông tin từ hóa đơn, hỗ trợ cho các sản phẩm khác của công ty, cụ thể là TPOS .
- Tìm kiếm sản phẩm tương tự là một tính năng của chatbot, cho phép người dùng có thể nhanh chóng tìm ra sản phẩm giống hoặc tương tự mà họ đang quan tâm bằng cách gửi một hình ảnh cho chatbot.
- Hệ thống chấm công bằng khuôn mặt sử dụng CCTV là một sản phẩm đột phá của team CV với mục đích hỗ trợ chấm công cho nhân viên của công ty một cách nhanh chóng (chỉ từ 1 - 2s nhìn vào camera). Sản phẩm cho phép người dùng mới đăng ký nhanh chóng với chỉ 5 bức ảnh ở các góc độ khác nhau. Điểm đặc biệt của hệ thống so với các hệ thống trên thị trường là nhân viên không cần phải đưa mặt gần camera mà chỉ cần ngước lên nhìn là có thể chấm công được.
Câu hỏi: Liệt kê các sản phẩm của team cv một cách ngắn gọn
"""
example = {"instruction": prompt, "input": input, "output": ""}
prompt = generate_prompt(example)
input_ids = tokenizer(prompt["text"], return_tensors="pt").input_ids
output = model.generate(input_ids=input_ids, generation_config=generation_config)
print(tokenizer.decode(output[0]))

<s> Dưới đây là một hướng dẫn mô tả một nhiệm vụ, kèm với thông tin cung cấp thêm bối cảnh.Hãy viết một phản hồi phù hợp để hoàn thành yêu cầu đó.

### Chỉ dẫn:
Dựa vào thông tin sau, hãy trả lời câu hỏi sau một cách ngắn gọn nhất. Không được phép xin lỗi

### Thông tin:

Thông tin:
Team CV có các sản phẩm như sau:
- Phân tích hình ảnh bao gồm việc nhận diện các vật thể trong hình và đưa ra các thông tin bao gồm: chủ thể, sản phẩm (áo, quần, dây nịt, ...) và các thông tin của sản phẩm như (màu sắc, kiểu dáng, ...)
- Lõi nhận diện chữ viết in là để nhằm trích xuất thông tin từ hóa đơn, hỗ trợ cho các sản phẩm khác của công ty, cụ thể là TPOS .
- Tìm kiếm sản phẩm tương tự là một tính năng của chatbot, cho phép người dùng có thể nhanh chóng tìm ra sản phẩm giống hoặc tương tự mà họ đang quan tâm bằng cách gửi một hình ảnh cho chatbot.
- Hệ thống chấm công bằng khuôn mặt sử dụng CCTV là một sản phẩm đột phá của team CV với mục đích hỗ trợ chấm công cho nhân viên của công ty một cách nhanh